In [3]:
import pandas as pd
import math as mt

In [4]:
event_training = pd.read_csv('training.csv')
event_test = pd.read_csv('test.csv')

# vamos a agregar otros features relacionados con el tiempo para cada usuario.

# vamos  a calcular  que tan lejos esta el primer evento de cada tipo para cada usuario con respecto a la fecha limite (31 de mayo del 2018 a las 23:59:59), este feature es similar al del paso 3

In [5]:
#primero agrego la columna date al set de datos
event_training['date'] = pd.to_datetime(event_training['timestamp'])

In [6]:
# A los que no realizaron alguno de los eventos les pongo -1, como manera de diferenciar

In [7]:
def calcularDistancia(dataframe, evento, nombre):
    fecha_limite=pd.to_datetime('2018-05-31 23:59:59')
    usuarios_con_evento = dataframe.groupby('person').filter(lambda x: (x['event']==evento).any())\
                    .reset_index(drop=True)
    usuarios_sin_evento = dataframe.groupby('person').filter(lambda x: (x['event']!=evento).all())\
                    .reset_index(drop=True)
    tabla = usuarios_sin_evento.loc[:,['person','timestamp']].drop_duplicates(subset='person',keep='first')
    tabla.columns=['person',nombre]
    tabla[nombre]=-1
    usuarios_con_evento = usuarios_con_evento.loc[usuarios_con_evento['event']==evento,['person','date']]\
                            .sort_values(by='date',ascending=True).groupby('person').min().reset_index()
    usuarios_con_evento[nombre]=(fecha_limite-usuarios_con_evento['date']).dt.total_seconds()
    tabla2=usuarios_con_evento.loc[:,['person',nombre]]
    tabla = pd.concat([tabla2,tabla])
    return tabla

In [8]:
first_vp = calcularDistancia(event_training,'viewed product','time_to_first_vp')
first_co = calcularDistancia(event_training,'conversion','time_to_first_co')
first_ch = calcularDistancia(event_training,'checkout','time_to_first_ch')
first_le = calcularDistancia(event_training,'lead','time_to_first_le')
first_ad = calcularDistancia(event_training,'ad compaign hit','time_to_first_ad')
first_bl = calcularDistancia(event_training,'brand listing','time_to_first_bl')
first_gl = calcularDistancia(event_training,'geberic listing','time_to_first_gl')
first_sp = calcularDistancia(event_training,'searched products','time_to_first_sp')
first_vs = calcularDistancia(event_training,'visited site','time_to_first_vs')
first_se = calcularDistancia(event_training,'shearch engine hit','time_to_first_se')
first_st = calcularDistancia(event_training,'staticpage','time_to_first_st')

In [10]:
first_time_train = first_vp.merge(first_co,on='person',how='inner').merge(first_ch,on='person',how='inner')\
                        .merge(first_le,on='person',how='inner').merge(first_ad,on='person',how='inner')\
                            .merge(first_bl,on='person',how='inner').merge(first_gl,on='person',how='inner')\
                                .merge(first_sp,on='person',how='inner').merge(first_vs,on='person',how='inner')\
                                    .merge(first_se,on='person',how='inner').merge(first_st,on='person',how='inner')

In [11]:
first_time_train.head()

,person,time_to_first_vp,time_to_first_co,time_to_first_ch,time_to_first_le,time_to_first_ad,time_to_first_bl,time_to_first_gl,time_to_first_sp,time_to_first_vs,time_to_first_se,time_to_first_st
0,000c79fe,257420.0,-1.0,257408.0,-1.0,-1.0,-1.0,-1.0,257515.0,257532.0,-1.0,-1.0
1,001802e4,23044.0,-1.0,22513.0,-1.0,-1.0,-1.0,-1.0,23021.0,23136.0,-1.0,-1.0
2,0019e639,10661605.0,9622600.0,9623036.0,-1.0,-1.0,10661625.0,-1.0,9622209.0,10661639.0,-1.0,-1.0
3,001b0bf9,812423.0,812173.0,812667.0,-1.0,-1.0,812435.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,001ca5ee,7609623.0,-1.0,1387354.0,-1.0,-1.0,3705816.0,-1.0,-1.0,7609625.0,-1.0,-1.0


# Ahora para el set de datos test:

In [12]:
event_test['date'] = pd.to_datetime(event_test['timestamp'])

In [14]:
first_vp_test = calcularDistancia(event_test,'viewed product','time_to_first_vp')
first_co_test = calcularDistancia(event_test,'conversion','time_to_first_co')
first_ch_test = calcularDistancia(event_test,'checkout','time_to_first_ch')
first_le_test = calcularDistancia(event_test,'lead','time_to_first_le')
first_ad_test = calcularDistancia(event_test,'ad compaign hit','time_to_first_ad')
first_bl_test = calcularDistancia(event_test,'brand listing','time_to_first_bl')
first_gl_test = calcularDistancia(event_test,'geberic listing','time_to_first_gl')
first_sp_test = calcularDistancia(event_test,'searched products','time_to_first_sp')
first_vs_test = calcularDistancia(event_test,'visited site','time_to_first_vs')
first_se_test = calcularDistancia(event_test,'shearch engine hit','time_to_first_se')
first_st_test = calcularDistancia(event_test,'staticpage','time_to_first_st')

In [15]:
first_time_test = first_vp_test.merge(first_co_test,on='person',how='inner').merge(first_ch_test,on='person',how='inner')\
                .merge(first_le_test,on='person',how='inner').merge(first_ad_test,on='person',how='inner')\
                    .merge(first_bl_test,on='person',how='inner').merge(first_gl_test,on='person',how='inner')\
                        .merge(first_sp_test,on='person',how='inner').merge(first_vs_test,on='person',how='inner')\
                            .merge(first_se_test,on='person',how='inner').merge(first_st_test,on='person',how='inner')

In [16]:
train_paso_5 = pd.read_csv('train_paso_5.csv')
test_paso_5 = pd.read_csv('test_paso_5.csv')

In [17]:
train_paso_6 = train_paso_5.merge(first_time_train,on='person',how='inner')
test_paso_6 = test_paso_5.merge(first_time_test,on='person',how='inner')

In [18]:
train_paso_6.head()

,person,label,mean_time_between_events,mean_time_between_co,mean_time_between_ch,mean_time_between_vs,mean_time_between_ad,mean_time_between_bl,mean_time_between_st,mean_time_between_se,...,time_to_first_co,time_to_first_ch,time_to_first_le,time_to_first_ad,time_to_first_bl,time_to_first_gl,time_to_first_sp,time_to_first_vs,time_to_first_se,time_to_first_st
0,0566e9c1,0,4.061890,0.0,0.000000,4.683844,4.197005,4.647535,0.0,0.000000,...,710133.0,710451.0,-1.0,-1.0,799392.0,-1.0,-1.0,799533.0,-1.0,631051.0
1,6ec7ee77,0,0.000000,-1.0,-1.000000,0.000000,0.000000,-1.000000,-1.0,-1.000000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,336875.0,-1.0,-1.0
2,abe7a2fb,0,4.992409,-1.0,0.000000,5.647888,5.933255,5.856175,-1.0,6.207509,...,-1.0,174045.0,-1.0,-1.0,9509437.0,-1.0,5710043.0,9509437.0,-1.0,-1.0
3,34728364,0,4.314423,-1.0,-1.000000,5.393602,-1.000000,2.422836,-1.0,0.000000,...,-1.0,-1.0,-1.0,-1.0,526988.0,-1.0,-1.0,1131498.0,-1.0,-1.0
4,87ed62de,0,1.944791,-1.0,2.187521,0.000000,2.534977,-1.000000,-1.0,-1.000000,...,-1.0,1124294.0,-1.0,-1.0,-1.0,-1.0,-1.0,1124607.0,-1.0,-1.0


In [19]:
test_paso_6.head()

,person,mean_time_between_events,mean_time_between_co,mean_time_between_ch,mean_time_between_vs,mean_time_between_ad,mean_time_between_bl,mean_time_between_st,mean_time_between_se,mean_time_between_sp,...,time_to_first_co,time_to_first_ch,time_to_first_le,time_to_first_ad,time_to_first_bl,time_to_first_gl,time_to_first_sp,time_to_first_vs,time_to_first_se,time_to_first_st
0,4886f805,2.239299,-1.0,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,...,-1.0,1208883.0,-1.0,-1.0,-1.0,-1.0,1208947.0,1209157.0,-1.0,-1.0
1,0297fc1e,4.325264,-1.0,6.252485,5.104949,5.601691,6.343117,-1.000000,-1.000000,6.177852,...,-1.0,11521119.0,4198650.0,-1.0,10596743.0,-1.0,12011361.0,12272605.0,-1.0,-1.0
2,2d681dd8,4.530487,-1.0,0.000000,5.928233,0.000000,5.326332,-1.000000,5.928233,0.000000,...,-1.0,361030.0,-1.0,-1.0,1209090.0,-1.0,361166.0,1209090.0,-1.0,-1.0
3,cccea85e,3.389035,-1.0,0.000000,4.988119,5.164210,5.521150,4.392996,4.912590,0.000000,...,-1.0,1730109.0,-1.0,-1.0,2078596.0,-1.0,40977.0,2078820.0,-1.0,1828920.0
4,4c8a8b93,3.130942,-1.0,4.701654,4.260423,4.295973,2.740926,-1.000000,4.253835,2.825507,...,-1.0,1165572.0,-1.0,-1.0,1130153.0,-1.0,1207872.0,1208085.0,-1.0,-1.0


In [20]:
test_paso_6.to_csv('test_paso_6.csv',index=False)
train_paso_6.to_csv('train_paso_6.csv',index=False)

In [21]:
train_paso_5.columns

Index(['most_frequent_event_0', 'most_frequent_event_1',
       'most_frequent_event_2', 'most_frequent_event_3',
       'most_frequent_event_4', 'less_frequent_event_0',
       'less_frequent_event_1', 'less_frequent_event_2',
       'less_frequent_event_3', 'less_frequent_event_4', 'person',
       'ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'mean_per_event',
       'count_most_frequent_event', 'count_less_frequent_event', 'label',
       '128GB', '16GB', '256GB', '32GB', '4GB', '512MB', '64GB', '8GB', 'Bom',
       'Bom - Sem Touch ID', 'Excelente', 'Muito Bom', 'Novo',
       'desconocido_modelo', 'Asus', 'LG', 'Lenovo', 'Motorola', 'Quantum',
       'Samsung', 'Sony', 'iPad', 'iPhone',
       'multiplicacion_checkout_y_conversion', 'suma_checkout_y_conversion',
       'lunes', 'martes', 'miercoles', 'jueves', 'viernes', 'sabado',
    